In [1]:
%load_ext autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from dbmaster import MasterParams, MasterPersist
from dbbase import MusicDBIDModVal, MusicDBDir, MusicDBData
from dbnote import DownloadRecord, MergeSearchArtist
from utils import FileIO, DirInfo, FileInfo, getFlatList, Timestat, TermTime, TermTimeTS, getTT
from pandas import Series, DataFrame, concat, Timestamp
from pandb import PanDBIO
from musicdb.youtubemusic import MusicDBParams, RawAPIData, MusicDBIO
from os import getpid
from functools import partial

mv = MusicDBIDModVal()
io = FileIO()
mpar = MusicDBParams()
dbio = MusicDBIO()
apiio = RawAPIData()
db = mpar.db

/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


In [3]:
searchArtistRecord = DownloadRecord(db=db, name="SearchArtist", rTypes=["Index", "Data"])
downloadArtistRecord = DownloadRecord(db=db, name="DownloadArtist", rTypes=["Index"])
downloadAlbumRecord = DownloadRecord(db=db, name="DownloadAlbum", rTypes=["Index"])
downloadSongRecord = DownloadRecord(db=db, name="DownloadSong", rTypes=["Index"])
knownArtists = dbio.rdio.getSummaryNameData
allArtists = partial(dbio.rdio.getData, "SearchArtist")

Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=YouTubeMusic, name=SearchArtist, rTypes=['Index', 'Data']
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=YouTubeMusic, name=DownloadArtist, rTypes=['Index']
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=YouTubeMusic, name=DownloadAlbum, rTypes=['Index']
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=YouTubeMusic, name=DownloadSong, rTypes=['Index']


In [ ]:
if False:
    from utils import FileInfo
    import shutil
    db = dbio.params.db
    dbdir = dbio.params.dir
    nameMap = {"MasterArtists": "SearchArtist", "LocalArtists": "DownloadArtist", "LocalAlbums": "DownloadAlbum"}
    nameMap = nameMap | {"LocalArtistTracks": "DownloadArtistTrack", "LocalArtistAlbums": "DownloadArtistAlbum",
                         "LocalArtistSimilar": "DownloadArtistSimilar", "LocalGenres": "DownloadGenre",
                         "LocalProperties": "DownloadProperty", "LocalArtistIDs": "DownloadArtistID"}
    dinfo = DirInfo(f"/Users/tgadfort/anaconda3/envs/post/pandb/musicdb/{db}")
    
    cpMap = {}
    for origname, newname in nameMap.items():
        cpMap[f"{dbdir}SearchedFor{origname}"] = f"{db}-{newname}-Index"
        cpMap[f"{dbdir}SearchedFor{origname}Data"] = f"{db}-{newname}-Data"
        cpMap[f"{dbdir}SearchedFor{origname}Errors"] = f"{db}-{newname}-Error"
    assert dinfo.exists(), f"{dinfo} does not exist!"
    for srcFile, dstFile in cpMap.items():
        srcFile = dinfo.join(f"{srcFile}.p")
        dstFile = dinfo.join(f"{dstFile}.p")
        if srcFile.exists():
            srcFile.mvFile(dstFile)

In [ ]:
##########################################################################################
# Show Summary
##########################################################################################
print(f"{db} Search Results (PID={getpid()})".format(db))
searchArtistRecord.info()
downloadArtistRecord.info()
downloadAlbumRecord.info()
downloadSongRecord.info()
print(f"  {'KnownArtist Data': <20}: {knownArtists().shape[0]}")

# Search For New Artists

In [5]:
######################################################################################################
# Explode Recent Data
######################################################################################################
def getNewData(searchArtistRecord):    
    newData = searchArtistRecord.getData()
    if len(newData) == 0:
        return DataFrame()
    newData = concat([DataFrame(s).T for s in newData.values()])
    newData.index = newData.index.astype(str)
    newData = newData[~newData.index.duplicated()].drop('id', axis=1)
    return newData

def mergeNewData(searchArtistRecord):
    searchArtistRecord.load(verbose=False)
    newData = getNewData(searchArtistRecord)
    msr = MergeSearchArtist(db)
    msr.mergeLocal(searchArtistRecord, newData, test=False)

In [8]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = PanDBIO().getUniqueUnmatchedArtistNames(db)
mergeNewData(searchArtistRecord)
searchedNames = Series(searchArtistRecord.getIndex())
artistNamesToGet = knownNames[~knownNames.isin(searchedNames.index)]

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {searchedNames.shape[0]}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del searchedNames
del knownNames

****************************************************************************************************************************
*                                            MergeSearchArtist(db=YouTubeMusic)                                            *
****************************************************************************************************************************
  ==> Creating copy of local SearchArtist data
  ==> Created copy of local SearchArtist data with (1415395, 5) shape
Found 13875 Searches
Found 18850 Results
Found 0 Duplicate Results
Found 1415395 Previously Known Results
Found 8736 New Results
  ==> 1.36 All Results / Search
  ==> 0.63 New Results / Search
Saving 1424131 Data ... Done
*****************************************************************************************************************************
*                                                 Initializing SearchArtist                                                 *
**********************************

In [7]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    apiio.sleep(sleeptime)
    
n = 0
maxN = 2500000
nErrors = []
searchArtistRecord.load(verbose=False)
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 5:
        print("Stopping due to 5 consecutive errors")
        break

    try:
        response = apiio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        isError(artistName, nErrors, 10, error)
        continue

    if not isinstance(response, dict):
        isError(artistName, nErrors, 3.5, "NotDict")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    apiio.sleep(3)
    n += 1
        
    if n % 5 == 0:
        if tt.isFinished():
            break
        apiio.sleep(1.0)
        
    if n % 100 == 0:
        ts.update(n=n)
        searchArtistRecord.save()
        apiio.wait(10.0)
        if tt.isFinished():
            break
    
    if n >= maxN:
        print(f"Breaking after {maxN} downloads...")
        break

ts.stop()
searchArtistRecord.save()

 [Getting YouTubeMusic ArtistIDs] | StartTime 2023-11-14 20:08:37
   ====> Terminate Time Set To 2023-11-15 10:50:00 <====
   ====> Will Terminate Process 14 Hour and 41 Minute From Now
Searching For Somebuddies                                       0
Searching For Ann Christin Ward                                 0
Searching For Dreamstate Seven                                  2
Searching For Renee Bondi                                       1
Searching For MaxTrax                                           0
Searching For TheOvale                                          15
Searching For jprecursor                                        0
Searching For Dummy Dumpster                                    1
Searching For Harry Caston                                      0
Searching For The Upfall Rockers                                0
Searching For Edmund Thomas Chipp                               0
Searching For Leonard Ritter                                    3
Searching For Luftrum

Waiting:   0%|          | 0/100 [00:00<?, ?it/s]

Searching For Pirates Of Liloa                                  1
Searching For Victor Chizhova                                   0
Searching For Kiba Dachi                                        0
Searching For Damaged Reality                                   1
Searching For Amy Cutler                                        0
Searching For Martin Enderle                                    0
Searching For Leseni Rogisti                                    0
Searching For Eww... You're a Girl and Girls Suck!              0
Searching For Mark Peru                                         1
Searching For Labyrinth Entrance                                0
Searching For Eva Tulkens                                       0
Searching For Trey Lambert                                      1
Searching For До Востребования                                  1
Searching For Gabto                                             2
Searching For Stefan Wulffraat                                  0
Searching 

Waiting:   0%|          | 0/100 [00:00<?, ?it/s]

Searching For Aziz Habibi                                       1
Searching For Dr Vulcano                                        0
Searching For Maria Grigorievna Poroshina                       0
Searching For Sinastra                                          1
Searching For Petite Storm                                      0
Searching For Jalal Chckara                                     0
Searching For Michiki Ono                                       0
Searching For Theodoro Christopher                              0
Searching For Kevin Thayer                                      0
Searching For Mimo Sound                                        1
Searching For Tom Mulcahy                                       0
Searching For Manfred Galland's vertagte Nächte                0
Searching For Swan Beats                                        2
Searching For Jānis Laucenieks                                 1
Searching For The VLP                                           1
Searching 

...

Searching For Cathal Ó Curráin                                0
Searching For Sonia Barros De Carusso                           0
Searching For Yuko Kimura                                       2
Searching For Deep Valley Blues                                 3
Searching For Mike Hendon                                       0
Searching For Jay Hale                                          0
Searching For Clubs & Spades                                    2
Searching For Brad Garton                                       1
Searching For you c                                             18
Searching For Of This Time                                      2
Searching For Augustin Foly                                     1
Searching For Moïse Ilunga                                     0
Searching For Olivia Mateu                                      0
Searching For Trioxide                                          1
Searching For Dungeon Wolf                                      2
Searching

Waiting:   0%|          | 0/100 [00:00<?, ?it/s]

Searching For James Doncsecz                                    0
Searching For Majora Philharmonic Sound Orchestra               0
Searching For Anna Wolińska                                    0
Searching For Newmen Thompson                                   0
Searching For Christin Cioma                                    1
Searching For Judith Wege                                       0
Searching For Bass Extreme                                      8
Searching For Roy Best                                          14
Searching For 中村舞子                                              4
Searching For Louis Zorich                                      1
Searching For Tsuruo Ogawa                                      0
Searching For Big Accident                                      2
Searching For Cheery Ciao Baby                                  1
Searching For Comao                                             2
Searching For Roland Debaets                                    0
Searching

Waiting:   0%|          | 0/100 [00:00<?, ?it/s]

Searching For Alfaomaga                                         1
Searching For The Motion Detectors                              1
Searching For N.O.R.A. alias T.N.                               0
Searching For Cleveland-Antwerp Connection                      0
Searching For Kenny Ausubel                                     0
Searching For Oleg Rasputinowitsch                              0
Searching For Laureate Sky                                      1
Searching For Caroline Buitenhuis                               0
Searching For Rudolf Kreis                                      0
Searching For Braziliers                                        1
Searching For Eminena                                           20
Searching For The Snuggle Ups                                   1
Searching For Ministros del Santuario                           1
Searching For Franco Masala                                     1
Searching For Tommy B. Paradise                                 1
Searching

In [ ]:
searchArtistRecord.save()

In [9]:
msa = MergeSearchArtist(dbio.db)
msa.copyFromLocal(force=True)

****************************************************************************************************************************
*                                            MergeSearchArtist(db=YouTubeMusic)                                            *
****************************************************************************************************************************
New Local Shape:  (1424131, 5)
Old Global Shape: (1174694, 5)
New Global Shape: (1424131, 5)


# Download Artist Data

In [4]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = allArtists()
knownNames['ModVal'] = knownNames.index.map(mv.getModVal)
availableNames = knownNames[~knownNames.index.map(downloadArtistRecord.isKnown)]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby("ModVal")})

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadArtistRecord.numKnown()}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

del availableNames
del knownNames

Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
# YouTubeMusic Search Results
#   Available Names:     920853
#   Known Artist Names:  998593
#   Artist Names To Get: 0


In [24]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistID, artistName, nErrors, sleeptime, error=None):
    downloadArtistRecord.setError(index=artistID)
    print(f"Artist Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    apiio.sleep(sleeptime)
    
n = 0
m = 0
maxN = 25000000
stop = False
nErrors = []
#downloadArtistRecord.load()
for modVal,modValData in artistNamesToGet.items():
    if stop is True:
        break
    N = modValData.shape[0]
    for i,(artistID,artistName) in enumerate(modValData['name'].items()):    
        if downloadArtistRecord.isKnown(artistID):
            continue
    
        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            break
    
        print(f"{modVal: <3} | {i: <6} | {N: <6} | {n: <6} | ", end="")
        
        try:
            response = apiio.getArtistData(artistName=artistName, artistID=artistID, getAlbums=True)
        except Exception as error:
            isError(artistID, artistName, nErrors, 5, error)
            continue
    
        if not isinstance(response, dict):
            isError(artistID, artistName, nErrors, 3.5, "NotDict")
            continue

        status     = response.get('Status')
        artistData = response.get('Artist')
        albumsData = response.get('Albums')
        if status is not True:
            pass  ## This happens because we're somehow searching for the browseId, 
                  ## which seems to match the albums Id for the artist, but does return the whole artist...

        nErrors = []
        if isinstance(artistData,dict):
            dbio.rdio.saveData("RawArtist", modVal, artistID, data=artistData)

        if isinstance(albumsData,list):
            dbio.rdio.saveData("RawArtistAlbum", modVal, artistID, data=albumsData)
            m += 1
            apiio.sleep(1.5)
        
        downloadArtistRecord.setIndex(index=artistID)
        apiio.sleep(3.0)
        n += 1
            
        if n % 5 == 0:
            if tt.isFinished():
                stop = True
                break
            
        if n % 25 == 0:
            ts.update(n=n)
            downloadArtistRecord.save()
            apiio.wait(10.0)
            if tt.isFinished():
                stop = True
                break
        
        if n >= maxN:
            print(f"Breaking after {maxN} downloads...")
            stop = True
            break

ts.stop()
downloadArtistRecord.save()

 [Getting YouTubeMusic ArtistIDs] | StartTime 2023-11-19 19:29:34
   ====> Terminate Time Set To 2023-11-20 10:50:00 <====
   ====> Will Terminate Process 15 Hour and 20 Minute From Now
 [Getting YouTubeMusic ArtistIDs] | RunTime 1 Second.  2023-11-19 19:29:35
*****************************************************************************************************************************
*                                                   Saving DownloadArtist                                                   *
*****************************************************************************************************************************
  Saving Index [745587] ... Done
  Saving Error [253006] ... Done


In [ ]:
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

# Download Album Data

## Create Media Data

In [ ]:
mediaData = {}
for modVal in range(100):
    modValData = dbio.data.getModValData(modVal)
    modValMediaData = {}
    for artistID,artistIDData in modValData.iteritems():
        for mediaType,mediaTypeData in artistIDData.media.media.items():
            modValMediaData.update({code: [artistID,media.album,media.url] for code,media in mediaTypeData.items()})
    mediaData.update(modValMediaData)
    if (modVal+1) % 10 == 0:
        print(f"ModVal = {modVal+1}")
        
df = DataFrame(mediaData).T
df.columns = ["ArtistID", "Name", "Ref"]
knownMedia.save(data=df)

## Download Data

In [ ]:
dbio   = bandcamp.MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = bandcamp.RawAPIData(debug=False)

In [ ]:
useArtist = False
numMaster = 250

knownAlbumsData = knownMedia.get()
knownAlbumsData['IndexModVal'] = knownAlbumsData.index.map(dbio.getModVal)
availableNames  = concat([artistIDDF.head(numMaster) for artistID,artistIDDF in knownAlbumsData.groupby(["ArtistID"])])
localAlbumsDict = localAlbums.get()
availableNames  = availableNames[~availableNames.index.isin(localAlbumsDict.keys())]
albumNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Album Search Results")
print(f"#   Available Album IDs:  {knownAlbumsData.shape[0]}")
print(f"#   Known Album IDs:      {len(localAlbumsDict)}")
print(f"#   Albums To Download:   {availableNames.shape[0]}")

del availableNames
del localAlbumsDict
del knownAlbumsData

#   Albums To Download:   23457
#   Albums To Download:   20437
#   Albums To Download:   12137
#   Albums To Download:   4132

In [ ]:
def saveAlbumData(db, localAlbumsDict, searchedForErrors):
    print("="*150)
    print(f"Saving {len(localAlbumsDict)} {db} Albums Data")
    localAlbums.save(data=localAlbumsDict)
    print(f"Saving {len(searchedForErrors)} {db} Searched For Errors")
    errors.save(data=searchedForErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} AlbumIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 25000000
localAlbumsDict     = localAlbums.get()
searchedForErrors   = errors.get()
stop = False
nErrors = []
for groupModVal,modValData in albumNamesToGet.iteritems():
    if stop is True:
        break
    for j,(albumID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(albumID) is not None for dct in [localAlbumsDict, searchedForErrors]]):
            continue
            
        artistID   = row["ArtistID"]
        albumName  = row["Name"]
        albumRef  = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = apiio.getAlbumData(albumName=albumName, albumRef=albumRef)
        except:
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            apiio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            apiio.sleep(3.5)
            continue

        nErrors = []
        modVal=dbio.mv.get(albumID)
        dbio.data.saveRawArtistAlbumData(data=response, modval=modVal, dbID=albumID)
        localAlbumsDict[albumID] = True
        apiio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            apiio.sleep(5)
            
        if n % 100 == 0:
            saveAlbumData(db, localAlbumsDict, searchedForErrors)
            if tt.isFinished():
                stop=True
                break
            apiio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveAlbumData(db, localAlbumsDict, searchedForErrors)

In [ ]:
from os import getpid
getpid()

# Download Lists

## Download Starter

In [ ]:
from apiutils import apiio
from ioutils import FileIO, HTMLIO
io  = FileIO()
hio = HTMLIO()
apiio = apiio()

In [ ]:
starter             = {}
starter["List"]     = "https://www.bandcamp.org/lists.php"
starter["Genre"]    = "https://www.bandcamp.org/genre.php"
#starterBestAlbum = "https://www.bandcamp.org/ratings/6-highest-rated/2023/1"
starter["Rating"]   = "https://www.bandcamp.org/ratings"
starter["Discover"] = "https://www.bandcamp.org/discover"
starter["Releases"] = "https://www.bandcamp.org/releases"
starter["MustHear"] = "https://www.bandcamp.org/must-hear"

savename = "../../sandbox/AOTYstarter.p"
starterData = io.get(savename)
for key,url in starter.items():
    if starterData.get(key) is not None:
        continue
    retval = apiio.get(url)
    if retval.code == 200:
        print(key)
        starterData[key] = retval.data
    apiio.sleep(3)
        
print(f"Saving data to {savename}")
io.save(idata=starterData, ifile=savename)

In [ ]:
useStarter = False
useSite    = True

if useStarter is True:
    aotyData = io.get("../../sandbox/AOTYstarter.p")
elif useSite is True:
    aotyData = io.get("../../sandbox/AOTYsiteData.p") | io.get("../../sandbox/AOTYsiteData2.p")
else:
    aotyData = {}
    
N  = len(aotyData)
ts = Timestat(f"Sorting {N} Site Refs")
refsData = {"List": {}, "Lists": {}, "Rating": {}, "Album": {}, "Artist": {}, "Discover": {}, "Genre": {}, "MustHear": {}, "Release": {}, "Spotify": {}, "Apple": {}, "Amazon": {}, "User": {}}
for n,(key,keyData) in enumerate(aotyData.items()):
    if (n+1) % 500 == 0 or (n+1) == 100:
        ts.update(n=n+1, N=N)
        
    bsdata = hio.get(keyData)
    refs = bsdata.findAll("a")
    for ref in refs:
        href = ref.get('href')
        if not isinstance(href,str):
            continue
        if href.startswith("/list/"):
            refsData["List"][href] = ref.text.strip()
        elif "lists.php" in href:
            refsData["Lists"][href] = ref.text.strip()
        elif href.startswith("/ratings/"):
            refsData["Rating"][href] = ref.text.strip()
        elif href.startswith("/artist/"):
            refsData["Artist"][href] = ref.text.strip()
        elif href.startswith("/album/"):
            refsData["Album"][href] = ref.text.strip()
        elif href.startswith("/discover/"):
            refsData["Discover"][href] = ref.text.strip()
        elif href.startswith("/genre/"):
            refsData["Genre"][href] = ref.text.strip()
        elif href.startswith("/must-hear/"):
            refsData["MustHear"][href] = ref.text.strip()
        elif "/releases/" in href:
            refsData["Release"][href] = ref.text.strip()
        elif "spotify.com" in href:
            refsData["Spotify"][href] = ref.text.strip()
        elif "apple.com" in href:
            refsData["Apple"][href] = ref.text.strip()
        elif "amazon.com" in href:
            refsData["Amazon"][href] = ref.text.strip()
        elif "/user/" in href:
            refsData["User"][href] = ref.text.strip()
        else:
            continue
            print(href,'\t|\t',ref.text)
            
ts.stop()

In [ ]:
savename = "../../sandbox/AOTYsiteRefs.p"
print(f"Saving data to {savename}")
for key,keyData in refsData.items():
    print(f"  {key: <20}{len(keyData)}")
io.save(idata=refsData, ifile=savename)

In [ ]:
siteData   = {}
artistData = {}
albumData  = {}
userData   = {}
otherData  = {}

if False:
    for ref,name in refsData["List"].items():
        if ref.startswith("/artist/"):
            artistData[ref] = name
        elif ref.startswith("/album/"):
            albumData[ref] = name
        elif ref.startswith("/user/"):
            userData[ref] = name
        elif ref.startswith("/"):
            siteData[ref] = name
        else:
            otherData[ref] = name
else:
    for key,keyData in refsData.items():
        for ref,name in keyData.items():
            if ref.startswith("/artist/"):
                artistData[ref] = name
            elif ref.startswith("/album/"):
                albumData[ref] = name
            elif ref.startswith("/user/"):
                userData[ref] = name
            elif ref.startswith("/"):
                if "/list/" in ref:
                    siteData[ref] = name
                else:
                    otherData[ref] = name
            else:
                otherData[ref] = name
            
print(f"Found {len(artistData)} Artist Refs")
print(f"Found {len(albumData)} Album Refs")
print(f"Found {len(userData)} User Refs")
print(f"Found {len(siteData)} Site Refs")
print(f"Found {len(otherData)} Other Refs")

In [ ]:
for year in range(1970,2005):
    key = f'/lists.php?y={year}'
    val = 'View More'
    siteData[key] = val

In [ ]:
savename1 = "../../sandbox/AOTYsiteData.p"
siteDataDownloads1 = io.get(savename1)
print(f"Found {len(siteDataDownloads1)} Previous Downloads")
savename2 = "../../sandbox/AOTYsiteData2.p"
siteDataDownloads2 = io.get(savename2)
print(f"Found {len(siteDataDownloads2)} Previous Downloads")
N = len(siteData)
ts = Timestat(f"Downloading {N} Site Refs")
for n,(ref,name) in enumerate(siteData.items()):
    url=f"https://www.bandcamp.org{ref}"
    if any([dct.get(ref) is not None for dct in [siteDataDownloads1,siteDataDownloads2]]):
        continue
    
    retval = apiio.get(url)
    if retval.code == 200:
        print(f"{n: <6} | {N: <6} | {ref}")
        siteDataDownloads2[ref] = retval.data
    apiio.sleep(3)
    
    if (n+1) % 25 == 0:
        ts.update(n=n+1,N=N)
        print(f"Saving {len(siteDataDownloads2)} data to {savename2}")
        io.save(idata=siteDataDownloads2, ifile=savename2)

ts.stop()
        
print(f"Saving data to {savename2}")
io.save(idata=siteDataDownloads2, ifile=savename2)

In [ ]:
siteDataDownloads = io.get("../../sandbox/AOTYsiteData.p")

In [ ]:
io.save(idata=siteDataDownloads, ifile=savename)

## Check For New Data

In [ ]:
from lib.bandcamp import MusicDBID
mid = MusicDBID()
mid.getAlbumID('/album/515536-beyonce-renaissance/critic-lists/?f=all&y=2022')

In [ ]:
df = DataFrame(Series(refsData["Album"])).reset_index().rename(columns={"index": "Ref", 0: "List"})
df["AlbumID"] = df["Ref"].map(mid.getAlbumID)
df = df[~df["AlbumID"].duplicated()]

In [ ]:
df = DataFrame(Series(refsData["Artist"])).reset_index().rename(columns={"index": "Ref", 0: "Name"})
df["ArtistID"] = df["Ref"].map(mid.getArtistID)
df = df[~df["ArtistID"].duplicated()]

In [ ]:
artistNames = searchArtists()

In [ ]:
df.index = df["ArtistID"]
df = df.drop(["ArtistID"], axis=1)
artistNames = concat([artistNames,df])
artistNames = artistNames[~artistNames.index.duplicated()]

In [ ]:
dbio.data.saveSearchArtistData(data=artistNames)

# Backup

In [ ]:
from utils import StoreData, backup
from numpy import array_split
sd = StoreData("bandcamp", "Artist")
for modVals in array_split(range(100), 2):
    sd.mergeLocalData(modVals=modVals)
sd.mergeGlobalData()

In [ ]:
from time import sleep
sleep(200)
sd.mergeGlobalData()

In [ ]:
for modVal in range(67):
    srcDir = DirInfo(f"/Volumes/Piggy/Discog/artists-bandcamp/{modVal}/artists")
    files  = [FileInfo(ifile) for ifile in srcDir.getFiles()]
    files  = [finfo for finfo in files if finfo.basename.isdigit()]
    dstDir = DirInfo(f"/Users/tgadfort/Music/Discog/artists-bandcamp/{modVal}/artists")
    for srcFile in files:
        dstFile = dstDir.join(srcFile.name)
        srcFile.mvFile(dstFile)